In [20]:
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec, clean_str
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine

import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Mecab
import openpyxl

In [21]:
# key_list = ['국문과제명','과제_한글키워드', '연구내용요약']
key_list = ['과제_한글키워드']

# regex = re.compile('[^ 가-힣0-9a-zA-Z!@#$%^&*.]|[가나다라마바사]\)|[0-9]-?[0-9]?\)')
regex = re.compile('[^ 가-힣a-zA-Z!@#$%^&*,]|[가나다라마바사]\)|[0-9]-?[0-9]?\)')
mecab = Mecab()


f = open('../../model/labels.txt', 'r')
labels = []
while True:
    line = f.readline().replace('\n', '')
    if not line:
        break
    labels.append(line)
f.close()

In [22]:
# tokenizer = BertTokenizer.from_pretrained('vocab.txt', do_lower_case=False) # 기존 vocab
# tokenizer = BertTokenizer.from_pretrained('../nscc_mecab_base_final/vocab.txt', do_lower_case=False) # 최신 vocab

def df_refine(df):
    # df = df.replace(',', ' ', regex=True)
    df = df.replace('[^ 가-힣0-9a-zA-Z!@#$%^&*.\n,]|[가나다라마바사]\)|[0-9]-?[0-9]?\)', '', regex=True)
    df = df[df['연구분야분류코드1']!='']
    df = df[df['과제_한글키워드']!='']
    df = df[df['연구내용요약']!='']
    df = df[df['연구분야분류코드1']!='-']
    df = df[df['과제_한글키워드']!='-']
    df = df[df['연구내용요약']!='-']
    df = df[df["과제_한글키워드"].str.len() > 1] # 키워드가 한 글자인 경우는 제외 (한 글자로는 옻, 배 2개가 있었음)
    df = df[df["과제_한글키워드"] != '보안'] # 키워드가 보안인 경우는 제외 (요약 내용이 제대로 나오지 않음)
    # df['과제_한글키워드'] = df['과제_한글키워드'].str.replace(' ', '')
    df = df.reset_index(drop=True)
    return df

# word에 한글과 영문 글자가 포함되어 있지 않으면 삭제 해주기
# 입력은 키워드(문자열)
# 반환은 bool, 한글과 영어가 포함된 키워드는 True를 반환, 한글과 영어가 포함되지 않은 키워드라면 False를 반환
def check_hangul(word):
    hangul_pattern = re.compile("["
                                u"\U0000AC00-\U0000D7AF"  # 한글 문자
                                u"\U00000041-\U0000007A"   # 영어 알파벳
                                "]+", flags=re.UNICODE)
    res = hangul_pattern.search(word)
    if res == None:
        return False
    else:
        return True

In [23]:
dir_path = '../jn'

summarys = []
keywords = []
codes = []
for file in os.listdir(dir_path):
    if 'xls' in file:
        print(os.path.join(dir_path, file))
        if file[-4:] == 'xlsx':
            df = openpyxl.load_workbook(os.path.join(dir_path, file))
            df = df.active
            df = pd.DataFrame(df.values)
            df = df.rename(columns=df.iloc[0])
            df = df.drop(df.index[0])
            df = df.reset_index(drop=True)
            df = df_refine(df)
        else:
            df = df_refine(pd.read_excel(os.path.join(dir_path, file)))
        for i in tqdm(range(len(df))):
            try:
                if '99' in df['연구분야분류코드1'].iloc[i]: # 분류되지 않은 태그는 제외
                    continue
            except:
                continue
            summary = df['연구내용요약'].iloc[i].replace(' ,', '').replace("\n", " ").replace("  ", " ")
            keyword = df['과제_한글키워드'].iloc[i].replace(' ,', '').replace(', ', ',')
            code = df['연구분야분류코드1'].iloc[i]
            if check_hangul(summary) and code[:4] in labels: # 키워드에 한글과 영어가 포함되어 있으면 추가
                summarys.append(summary)
                keywords.append(keyword)
                codes.append(code[:4])
            
print("summarys : {}, codes : {}".format(len(summarys), len(codes)))

../jn/180830_사업정보_2014_조분Y_송부.xls


100%|██████████| 44303/44303 [00:02<00:00, 20336.21it/s]


../jn/180830_사업정보_2015_조분Y_송부.xls


100%|██████████| 45332/45332 [00:02<00:00, 20248.16it/s]


../jn/180830_사업정보_2016_조분Y_송부.xls


...

../jn/2019년사업과제정보_rev.xlsx


100%|██████████| 64224/64224 [00:03<00:00, 21152.08it/s]


../jn/2018년사업과제정보_rev.xls


100%|██████████| 55422/55422 [00:02<00:00, 20481.55it/s]

summarys : 235827, codes : 235827


In [24]:
print(len(keywords))
print(len(summarys))
print(len(codes))

235827
235827
235827


In [25]:
summarys[:5]

['Ground reaction forceG.R.F. 측정 센서의 multi axis 적용을 위한 메커니즘 설계 및 제작 유니버셜 스테이지의 축변환에 대응하여 변화되는 G.R.F.를 실시간 획득하는 기술개발 고령자의 동작 인식을 위해 융합 Depth sensor 개발 유니버셜 스테이지의 움직임에 대응하여 변화되는 G.R.F.를 실시간으로 획득하는 기술개발 시스템 통합을 위한 API 개발 다물체 동역학 시뮬레이션과 삼차원 모션캡쳐를 통한 시스템 작동 유효성 검증',
 ' 고령자의 운동 형태별 모션 측정을 통한 균형 능력 향상에 영향을 미치는 동작 인자 추출 근전도를 활용한 고령자의 근골격계 능력 향상에 영향을 미치는 동작 인자 추출 신체 기능 훈련의 형태, 종류, 방향 등을 고려하여 고령자의 근골격계 및 균형 능력 향상을 위한 훈련 콘텐츠 개발 콘텐츠의 운동 효과 분석을 위한 시각화 알고리즘 개발 유니버셜 스테이지 통합 시스템의 기계적전기적 안전성 평가 및 분석 고령자를 대상으로 유니버셜 스테이지 통합 시스템의 사용성 평가 및 분석',
 '1단계 1년차 13년 피부로 호르몬 투여 시에 뇌기능 조절인자 및 피부노화 조절인자의 변화를 관찰함  생쥐에서 외부로부터 투여한 호르몬에 의한 뇌기능 개선 효과 규명  생쥐에서 외부로부터 투여한 호르몬에 의한 피부노화 개선 효과 규명  1단계 2년차 14년 후보물질을 섭취한 동물 뇌 조직에서 해마부위의 신경세포 변화 및 뇌기능 조절 인자 BDNF 등의 발현 변화를 관찰함 생쥐에서 탐색된 건강기능식품의 식이에 의한 뇌기능 개선 효과 규명 효능물질의 뇌기능 개선 기전을 조직학, 분자생물학적인 수준에서 기전 연구 1단계 3년차 15년 후보물질을 섭취한 동물의 피부에서 피부노화관련인자 collagenMMPs등의 발현을 확인하고, 자외선에 의한 주름살 억제효능을 증명함 생쥐에서 탐색된 건강기능식품의 식이에 의한 피부노화 개선 효과 규명 효능물질의 피부기능 개선 기전을 조직학, 분자생물학적인 수준에서 기전 연구',
 '1단계 1년차 13

In [26]:
sentences = summarys

In [27]:
#!/usr/bin/python
#-*-coding:utf-8-*-

dataset_name = 'NTIS'
# sentences = ['Would you like a plain sweater or something else?', 'Great. We have some very nice wool slacks over here. Would you like to take a look?']
labels = ['Yes' , 'No']
train_or_test_list = ['train', 'test']


meta_data_list = []

for i in tqdm(range(len(sentences))):
    if i / len(sentences) < 0.8:
        meta = str(i) + '\t' + train_or_test_list[0] + '\t' + codes[i]
        meta_data_list.append(meta)
    else:
        meta = str(i) + '\t' + train_or_test_list[1] + '\t' + codes[i]
        meta_data_list.append(meta)

meta_data_str = '\n'.join(meta_data_list)

f = open('data/' + dataset_name + '.txt', 'w')
f.write(meta_data_str)
f.close()

corpus_str = '\n'.join(sentences)

f = open('data/corpus/' + dataset_name + '.txt', 'w')
f.write(corpus_str)
f.close()

100%|██████████| 235827/235827 [00:00<00:00, 752710.53it/s]


In [28]:
corpus_str[:10000]
sentences

['Ground reaction forceG.R.F. 측정 센서의 multi axis 적용을 위한 메커니즘 설계 및 제작 유니버셜 스테이지의 축변환에 대응하여 변화되는 G.R.F.를 실시간 획득하는 기술개발 고령자의 동작 인식을 위해 융합 Depth sensor 개발 유니버셜 스테이지의 움직임에 대응하여 변화되는 G.R.F.를 실시간으로 획득하는 기술개발 시스템 통합을 위한 API 개발 다물체 동역학 시뮬레이션과 삼차원 모션캡쳐를 통한 시스템 작동 유효성 검증',
 ' 고령자의 운동 형태별 모션 측정을 통한 균형 능력 향상에 영향을 미치는 동작 인자 추출 근전도를 활용한 고령자의 근골격계 능력 향상에 영향을 미치는 동작 인자 추출 신체 기능 훈련의 형태, 종류, 방향 등을 고려하여 고령자의 근골격계 및 균형 능력 향상을 위한 훈련 콘텐츠 개발 콘텐츠의 운동 효과 분석을 위한 시각화 알고리즘 개발 유니버셜 스테이지 통합 시스템의 기계적전기적 안전성 평가 및 분석 고령자를 대상으로 유니버셜 스테이지 통합 시스템의 사용성 평가 및 분석',
 '1단계 1년차 13년 피부로 호르몬 투여 시에 뇌기능 조절인자 및 피부노화 조절인자의 변화를 관찰함  생쥐에서 외부로부터 투여한 호르몬에 의한 뇌기능 개선 효과 규명  생쥐에서 외부로부터 투여한 호르몬에 의한 피부노화 개선 효과 규명  1단계 2년차 14년 후보물질을 섭취한 동물 뇌 조직에서 해마부위의 신경세포 변화 및 뇌기능 조절 인자 BDNF 등의 발현 변화를 관찰함 생쥐에서 탐색된 건강기능식품의 식이에 의한 뇌기능 개선 효과 규명 효능물질의 뇌기능 개선 기전을 조직학, 분자생물학적인 수준에서 기전 연구 1단계 3년차 15년 후보물질을 섭취한 동물의 피부에서 피부노화관련인자 collagenMMPs등의 발현을 확인하고, 자외선에 의한 주름살 억제효능을 증명함 생쥐에서 탐색된 건강기능식품의 식이에 의한 피부노화 개선 효과 규명 효능물질의 피부기능 개선 기전을 조직학, 분자생물학적인 수준에서 기전 연구',
 '1단계 1년차 13